In [1]:
pip install streamlit requests opencv-python numpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import streamlit as st
import cv2
import requests
import numpy as np
from PIL import Image
import base64

st.title("Webcam-based Deep Learning App")

# OpenCV webcam capture
cap = cv2.VideoCapture(0)
frame_placeholder = st.empty()

# Stop button
stop = st.button("Stop Webcam")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret or stop:
        st.write("Stopping webcam...")
        break

    # Convert frame to PIL Image
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    frame_placeholder.image(img, caption="Live Webcam Feed", use_column_width=True)

    # Encode image and send to backend
    _, encoded_image = cv2.imencode(".jpg", frame)
    response = requests.post("http://127.0.0.1:8000/process/", 
                             files={"file": ("image.jpg", encoded_image.tobytes(), "image/jpeg")})

    if response.status_code == 200:
        processed_image_str = response.json()["processed_image"]
        processed_image_bytes = base64.b64decode(processed_image_str)
        processed_image = np.frombuffer(processed_image_bytes, np.uint8)
        processed_image = cv2.imdecode(processed_image, cv2.IMREAD_COLOR)
        st.image(processed_image, caption="Processed Frame", use_column_width=True)

cap.release()


2025-03-21 15:30:04.871 
  command:

    streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
